# Artificial and Computational Intelligence: Assignment 1

#### Assignment Contributor


This assignment is submitted by: **Group 306**.

The team includes:

| S.no.          | Name           | BITS ID        | Contribution   |
|----------------|----------------|----------------|----------------|
| 1              | Vidushi Bhatia | 2024ac05012    | 100%           |

<br>

<hr>

<br>

### Problem Overview

<u>**PEAS Description**</u>

**P - Performance Metrics**
- Total path cost - Use the most efficient (optimal) path according to the defined costs. (minimize)
- Number of collisions with walls. (minimize)
- Number of steps to reach the finish point. (minimize)
- Number of steps taken in the east direction - towards the sun. (minimize)
- Time taken to find the finish position. (minimize)

**E - Environment**
- A maze with gird layout.
- Clearly defined start and goal positions.
- Fixed walls that act as obstacles and block movement.
- Sunlight exposure on the east.
- Static, observable layout.
- No dynamic obstacles, deterministic.


**A - Actuators**
- Ability to traverse on open paths - robot legs/wheels.
- Ability to change direction / steering ability. (4 directions)
- Wall-avoidance mechanism (halt on blocked direction).

**S - Sensors**
- Sensors to detect walls (Proximity/vision).
- Sensors to dectect sunlight (thermal/UV).
- Position awareness within the grid (GPS/odometry).

>
> **Note:**  <br> Detailed theoretical information about the problem can be found in the Google Docs. This notebook only covers the coding portion.

<hr>

### Problem Solution

In [2]:
import time
import heapq
import sys
import tracemalloc

#### 1. Util Functions for Initial State, Path Cost Computation and Successors¶

In [13]:
# Set Initial state with dynamic parameters

def is_valid(pos, rows, cols):
        return 0 <= pos[0] < rows and 0 <= pos[1] < cols and pos in maze

def set_initial_state(rows, cols, maze, start_input, goal_input):    
    if not is_valid(start_input, rows, cols):
        raise ValueError(f"Invalid start position: {start_input}")
    if not is_valid(goal_input, rows, cols):
        raise ValueError(f"Invalid goal position: {goal_input}")
    if start_input == goal_input:
        raise ValueError("Start and goal cannot be the same.")

    return {
        'start': start_input,
        'goal': goal_input,
        'current': start_input
    }

In [4]:
# define heuristic function
def manhattan(current, goal):
    return abs(current[0] - goal[0]) + abs(current[1] - goal[1])


In [5]:
# define function to get successors and compute path cost

def get_successors(cell, adj_map, goal):
    x, y = cell
    directions = {
        0: (-1, 0),  # North
        1: (1, 0),   # South
        2: (0, 1),   # East (sunlight penalty)
        3: (0, -1)   # West
    }

    successors = []
    for i in range(4):
        if adj_map[cell][i]:  # direction is open
            dx, dy = directions[i]
            neighbor = (x + dx, y + dy)
            if neighbor in adj_map:
                move_cost = 3 + (5 if i == 2 else 0)  # East penalty
                successors.append((neighbor, move_cost))
    return successors

In [6]:
# check if goal state is achieved

def is_goal(state, goal):
    return state == goal



#### 2. Function for A* Algorithm Implementation

In [7]:
def a_star_search(adj_map, start, goal, w=1.0):
    open_list = []
    heapq.heappush(open_list, (0, 0, start))  # (f, g, state)
    came_from = {}
    cost_so_far = {start: 0}
    nodes_expanded = 0

    while open_list:
        print(f"\n--- Step {nodes_expanded + 1} ---")
        print("Open List:")
        for f, g, node in open_list:
            print(f"  State: {node}, g(n): {g}, h(n): {manhattan(node, goal)}, f(n): {f:.1f}")

        _, g, current = heapq.heappop(open_list)
        nodes_expanded += 1
        print(f"\nExpanding: {current}")

        if is_goal(current, goal):
            path = reconstruct_path(came_from, current)
            return {
                'path': path,
                'cost': cost_so_far[current],
                'nodes_expanded': nodes_expanded
            }

        for neighbor, move_cost in get_successors(current, adj_map, goal):
            new_cost = cost_so_far[current] + move_cost
            if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = new_cost
                h = manhattan(neighbor, goal)
                f = new_cost + w * h
                heapq.heappush(open_list, (f, new_cost, neighbor))
                came_from[neighbor] = current
                print(f"  → Considering: {neighbor}, g(n): {new_cost}, h(n): {h}, f(n): {f:.1f}")

    return None

In [8]:
def reconstruct_path(came_from, current):
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    return path[::-1]

<hr>

#### 4. Dynamic Run - Define Maze & call Initial State

The maze is imagined as a **grid structure** with n rows and m cols. Each cell in this grid can have walls on **4 sides** which can obstruct the path. 

To define this maze, a dictionary is used with the format: <br>

        
        { 
        (row, col): [N, S, E, W],
        ... 
        }
In this dictionary,    
- `(row, col)` is a tuple showcasing the position of the current cell w.r.t the grid. The first row is 0, and the first column is also taken as 0.
- `[N, S, E, W]` is a list associated with each cell position showcasing the presence or absence of walls and hence the path. The direction layout is kept constant and boolean 1 and 0 are used to represent the presence of open path or wall respectively.

e.g. 

`(2,3): [1,0,1,0]` means in the cell (2,3) :
- north is open.
- south is closed.
- east is open.
- west is closed.

Hence, the agent can move up and right.

<br>


In [14]:
## MAZE INPUT
# format of the dictionary is (row, col): [N, S, E, W] where 1 means open path and 0 means blocked.
# e.g. (2,3): [1,0,1,0] means in the cell (2,3) north and east are open, hence the agent can move up and right.

maze = {
    (0, 0): [0, 1, 1, 0],
    (0, 1): [0, 1, 1, 1],
    (0, 2): [0, 1, 0, 1],
    (0, 3): [0, 1, 1, 0],
    (0, 4): [0, 1, 0, 1],
    (0, 5): [0, 1, 1, 0],
    (0, 6): [0, 1, 0, 1],

    (1, 0): [1, 1, 0, 0],
    (1, 1): [1, 1, 0, 0],
    (1, 2): [1, 1, 0, 0],
    (1, 3): [1, 1, 0, 0],
    (1, 4): [1, 1, 0, 0],
    (1, 5): [1, 1, 0, 0],
    (1, 6): [1, 1, 0, 0],

    (2, 0): [1, 0, 0, 0],
    (2, 1): [1, 1, 0, 0],
    (2, 2): [1, 1, 0, 0],
    (2, 3): [1, 1, 0, 0],
    (2, 4): [1, 1, 0, 0],
    (2, 5): [1, 1, 0, 0],
    (2, 6): [1, 1, 0, 0],

    (3, 0): [0, 1, 1, 1], # START
    (3, 1): [1, 0, 0, 1],
    (3, 2): [1, 1, 0, 0],
    (3, 3): [1, 1, 0, 0],
    (3, 4): [1, 1, 0, 0],
    (3, 5): [1, 1, 0, 0],
    (3, 6): [1, 1, 1, 0], # FINISH

    (4, 0): [1, 1, 0, 0],
    (4, 1): [0, 0, 1, 0],
    (4, 2): [1, 0, 1, 1],
    (4, 3): [1, 0, 0, 1],
    (4, 4): [1, 0, 1, 0],
    (4, 5): [1, 0, 0, 1],
    (4, 6): [1, 1, 0, 0],

    (5, 0): [1, 0, 1, 0],
    (5, 1): [0, 0, 1, 1],
    (5, 2): [0, 0, 1, 1],
    (5, 3): [0, 0, 1, 1],
    (5, 4): [0, 0, 1, 1],
    (5, 5): [0, 0, 1, 1],
    (5, 6): [1, 0, 0, 1]
}

In [16]:
total_rows = 6
total_cols = 7
start = (3, 0)
goal = (3, 6)
initial_state = set_initial_state(total_rows, total_cols, maze, start, goal)

#### 5. Run A* search algorithm with Dynamic Input

**Scenario 1**
- Heuristic: Manhattan Distance
- Heuristic Weight (w): 1.0
- Observation Focus: Optimal path guaranteed, slower due to full exploration.

In [17]:
tracemalloc.start()
start_time = time.time()
result_1 = a_star_search(
    adj_map=maze,
    start=initial_state['start'],
    goal=initial_state['goal'],
    w=1.0,
)
end_time = time.time()
execution_time_1 = end_time - start_time
current_1, peak_1 = tracemalloc.get_traced_memory()
tracemalloc.stop()


--- Step 1 ---
Open List:
  State: (3, 0), g(n): 0, h(n): 6, f(n): 0.0

Expanding: (3, 0)
  → Considering: (4, 0), g(n): 3, h(n): 7, f(n): 10.0
  → Considering: (3, 1), g(n): 8, h(n): 5, f(n): 13.0

--- Step 2 ---
Open List:
  State: (4, 0), g(n): 3, h(n): 7, f(n): 10.0
  State: (3, 1), g(n): 8, h(n): 5, f(n): 13.0

Expanding: (4, 0)
  → Considering: (5, 0), g(n): 6, h(n): 8, f(n): 14.0

--- Step 3 ---
Open List:
  State: (3, 1), g(n): 8, h(n): 5, f(n): 13.0
  State: (5, 0), g(n): 6, h(n): 8, f(n): 14.0

Expanding: (3, 1)
  → Considering: (2, 1), g(n): 11, h(n): 6, f(n): 17.0

--- Step 4 ---
Open List:
  State: (5, 0), g(n): 6, h(n): 8, f(n): 14.0
  State: (2, 1), g(n): 11, h(n): 6, f(n): 17.0

Expanding: (5, 0)
  → Considering: (5, 1), g(n): 14, h(n): 7, f(n): 21.0

--- Step 5 ---
Open List:
  State: (2, 1), g(n): 11, h(n): 6, f(n): 17.0
  State: (5, 1), g(n): 14, h(n): 7, f(n): 21.0

Expanding: (2, 1)
  → Considering: (1, 1), g(n): 14, h(n): 7, f(n): 21.0

--- Step 6 ---
Open List:


In [20]:
if result_1:
    print("Final Solution:")
    print("--> Path:")
    for step in result_1['path']:
        print(f"   {step}")
    print(f"--> Total Cost: {result_1['cost']}")
    print(f"--> Total Steps: {len(result_1['path']) - 1}")
    print(f"--> Nodes Expanded: {result_1['nodes_expanded']}")
else:
    print("No path found.")

Final Solution:
--> Path:
   (3, 0)
   (4, 0)
   (5, 0)
   (5, 1)
   (5, 2)
   (5, 3)
   (5, 4)
   (5, 5)
   (5, 6)
   (4, 6)
   (3, 6)
--> Total Cost: 60
--> Total Steps: 10
--> Nodes Expanded: 28


<hr> 

**Scenario 2**
- Heuristic: Manhattan Distance
- Heuristic Weight (w): 2.5
- Observation Focus: Faster search with fewer nodes expanded, but path may be suboptimal



In [21]:
tracemalloc.start()
start_time = time.time()
result_2 = a_star_search(
    adj_map=maze,
    start=initial_state['start'],
    goal=initial_state['goal'],
    w=2.5
)
end_time = time.time()
execution_time_2 = end_time - start_time
current_2, peak_2 = tracemalloc.get_traced_memory()
tracemalloc.stop()


--- Step 1 ---
Open List:
  State: (3, 0), g(n): 0, h(n): 6, f(n): 0.0

Expanding: (3, 0)
  → Considering: (4, 0), g(n): 3, h(n): 7, f(n): 20.5
  → Considering: (3, 1), g(n): 8, h(n): 5, f(n): 20.5

--- Step 2 ---
Open List:
  State: (4, 0), g(n): 3, h(n): 7, f(n): 20.5
  State: (3, 1), g(n): 8, h(n): 5, f(n): 20.5

Expanding: (4, 0)
  → Considering: (5, 0), g(n): 6, h(n): 8, f(n): 26.0

--- Step 3 ---
Open List:
  State: (3, 1), g(n): 8, h(n): 5, f(n): 20.5
  State: (5, 0), g(n): 6, h(n): 8, f(n): 26.0

Expanding: (3, 1)
  → Considering: (2, 1), g(n): 11, h(n): 6, f(n): 26.0

--- Step 4 ---
Open List:
  State: (5, 0), g(n): 6, h(n): 8, f(n): 26.0
  State: (2, 1), g(n): 11, h(n): 6, f(n): 26.0

Expanding: (5, 0)
  → Considering: (5, 1), g(n): 14, h(n): 7, f(n): 31.5

--- Step 5 ---
Open List:
  State: (2, 1), g(n): 11, h(n): 6, f(n): 26.0
  State: (5, 1), g(n): 14, h(n): 7, f(n): 31.5

Expanding: (2, 1)
  → Considering: (1, 1), g(n): 14, h(n): 7, f(n): 31.5

--- Step 6 ---
Open List:


In [23]:
if result_2:
    print("Final Solution:")
    print("--> Path:")
    for step in result_2['path']:
        print(f"   {step}")
    print(f"--> Total Cost: {result_2['cost']}")
    print(f"--> Total Steps: {len(result_2['path']) - 1}")
    print(f"--> Nodes Expanded: {result_2['nodes_expanded']}")
else:
    print("No path found.")

Final Solution:
--> Path:
   (3, 0)
   (4, 0)
   (5, 0)
   (5, 1)
   (5, 2)
   (5, 3)
   (5, 4)
   (5, 5)
   (5, 6)
   (4, 6)
   (3, 6)
--> Total Cost: 60
--> Total Steps: 10
--> Nodes Expanded: 26


#### 6. Time and Space Computation

In [24]:
## Time Taken
print(f"Time Taken for first run: {execution_time_1:.6f} seconds")
print(f"Time Taken for first run: {execution_time_2:.6f} seconds")


Time Taken for first run: 0.012563 seconds
Time Taken for first run: 0.010879 seconds


In [25]:
## Peak memory consumed
print(f"Peak Memory Usage for first run: {peak_1 / 1024:.2f} KB")
print(f"Peak Memory Usage for second run: {peak_2 / 1024:.2f} KB")


Peak Memory Usage for first run: 48.81 KB
Peak Memory Usage for second run: 47.43 KB


<hr>